<a href="https://colab.research.google.com/github/Razanhus/week9/blob/main/Rzn_RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

In [2]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 410kB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [3]:
import pandas as pd

In [ ]:
!pip install langchain langchain_community

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain_groq

In [46]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq


In [ ]:
!pip install transformers

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [4]:
df = pd.read_csv("/content/Dataset.csv")

In [ ]:
pd.set_option('display.max_rows', None)
display(df)

In [6]:
df.shape

(104, 2)

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [13]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [18]:
for i in range(0, len(df)):

    violation = df['المخالفة'].iloc[i]
    fine = df['الغرامة'].iloc[i]

    markdown_content = f"# {violation}\n\n"
    markdown_content += f"{fine}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [19]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [32]:
embedding_function = SentenceTransformerEmbeddings(model_name="aubmindlab/bert-base-arabertv02")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 7: Initialize the Language Model

In [41]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [48]:
PROMPT_TEMPLATE = """
استنادًا إلى المعلومات التالية فقط، قم بالإجابة على السؤال:
المعلومات: {context}
السؤال: {question}
إجابتك:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context","question"]
)

In [49]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

<ipython-input-49-a2932d0d6a9b>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=llm,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [50]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [73]:
def inference_pipeline():
    user_query = input("ايش سؤالك):؟")
    response = query_rag(user_query)
    print("الجواب:", response)

inference_pipeline()

ايش سؤالك):؟ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟"


Prompt after formatting:

استنادًا إلى المعلومات التالية فقط، قم بالإجابة على السؤال:
المعلومات: <h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>
<p>الغرامة المالية 500 - 900 ريال</p>

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>تسيير مركبات الأشغال العامة على الطرق قبل اتخاذ الإجراءات اللازمة لتلافي أضرارها، بما في ذلك عدم وضع الشرائح العاكسة على جانبي مؤخرة المركبة.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>
السؤال: ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟"
إجابتك:


> Finished chain.
الجواب: {'context': '<h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>\n\n<h1>قيادة المركبة برخصة قيادة لا تتناسب 

In [51]:
rows = df[df['المخالفة'] == 'استخدام لوحات غير عائدة للمركبة.']
print(rows)

                             المخالفة                            الغرامة
101  استخدام لوحات غير عائدة للمركبة.  الغرامة المالية 5000 - 10000 ريال


In [53]:

rows = df[df['المخالفة'] == 'سير المركبة بلا لوحة خلفية، أو بلا لوحات.']
print(rows)

                                     المخالفة  \
96  سير المركبة بلا لوحة خلفية، أو بلا لوحات.   

                             الغرامة  
96  الغرامة المالية 3000 - 6000 ريال  


In [52]:
response = query_rag("ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟")
response

Prompt after formatting:

استنادًا إلى المعلومات التالية فقط، قم بالإجابة على السؤال:
المعلومات: <h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>
<p>الغرامة المالية 500 - 900 ريال</p>

<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز.</h1>
<p>الغرامة المالية 500 - 900 ريال</p>
السؤال: ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟
إجابتك:


> Finished chain.


{'context': '<h1>عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>\n\n<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>\n\n<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>\n\n<h1>عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>',
 'question': 'ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟',
 'text': 'الغرامة المالية 1000 - 2000 ريال.'}

In [ ]:
response = query_rag("ماهي الغرامة على التجمهر في موقع الحادث؟")
response

In [66]:
query = "ماهي الغرامة على التجمهر في موقع الحادث؟"
similarity_search_results = db.similarity_search_with_score(query, k=4)

In [70]:
print("First: ", similarity_search_results[0][0].page_content)
print("Second: ", similarity_search_results[1][0].page_content)
print("Third: ", similarity_search_results[2][0].page_content)
print("Fourth: ", similarity_search_results[3][0].page_content)

First:  <h1>عدم إعطاء الأفضلية للمركبات التي بداخل الدوار من قبل المركبات التي خارجه في حالة عدم وجود إشارات ضوئية أو رجل أمن يوجه السير.</h1>
<p>الغرامة المالية 500 - 900 ريال</p>
Second:  <h1>قيام السائق بتخطي أرتال المركبات أمام إشارات المرور أو نقاط الضبط الأمني باستخدام كتف الطريق أو المسار المخصص للالتفاف.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>
Third:  <h1>تركيب تجهيزات في المركبة كتلك الخاصة بالمركبات الرسمية ومركبات الطوارئ.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>
Fourth:  <h1>تسيير مركبات الأشغال العامة على الطرق قبل اتخاذ الإجراءات اللازمة لتلافي أضرارها، بما في ذلك عدم وضع الشرائح العاكسة على جانبي مؤخرة المركبة.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>


In [71]:
print(similarity_search_results[0][1])
print(similarity_search_results[1][1])
print(similarity_search_results[2][1])
print(similarity_search_results[3][1])

202.95266723632812
206.18600463867188
207.601806640625
208.4356231689453
